In [1]:
import numpy as np
import alc

In [3]:
# Cargar datos 
train_cats = np.load('cats_and_dogs\\train\\cats\\efficientnet_b3_embeddings.npy')
train_dogs = np.load('cats_and_dogs\\train\\dogs\\efficientnet_b3_embeddings.npy')

val_cats = np.load('cats_and_dogs\\val\\cats\\efficientnet_b3_embeddings.npy')
val_dogs = np.load('cats_and_dogs\\val\\dogs\\efficientnet_b3_embeddings.npy')


(1536, 1000)